In [1]:
import owlready2 as owl
import csv
import os
owl.onto_path.append("../ontology")
ontology = owl.get_ontology('file://../ontology/put-dataset-data-lineage-individuals-base.rdf')
ontology.load(format='rdfxml')

data_dir = '../results_csv/multi_scenario'
def parse_name(table_name):
    name = table_name.strip()
    name = name.replace('#','')
    name = name.replace('_','')
    name = name.replace(' ','')
    name = name.strip('[')
    name = name.strip(']')
    name = name.replace('.csv', '')
    return name
for directory in os.listdir(data_dir):
    print(directory)
    print(len(list(ontology.individuals())))
    if directory.startswith('lineage'):
        continue
    with open(data_dir + '/' + directory, newline='') as csvfile:
        tableName = parse_name(directory)
        onto_table = ontology.Table(tableName)
        onto_table.label = [tableName]
        onto_table.title = [tableName]
        
        columns = [column.strip() for column in csvfile.readline().split(';')]
        onto_columns = []
        for column in columns:
            onto_column = ontology.Column(column)
            onto_column.label = [column,]
            onto_table.hasColumn.append(onto_column)
            onto_column.isColumnOf.append(onto_table)
            onto_columns.append(onto_column)
            
        reader = csv.reader(csvfile, delimiter=';')
        rownum = 1
        for row in reader:
            onto_row = ontology.Row()
            onto_row.rownum.append(rownum)
            rownum += 1
            for colno,value in enumerate(row):
                onto_value = ontology.CellValue()
                onto_value.exactValue = [value]
                onto_row.hasCellValue.append(onto_value)
                onto_value.belongsToColumn.append(onto_columns[colno])
                onto_columns[colno].hasInstance.append(onto_value)
            onto_table.hasRow.append(onto_row)


Customers.csv
0
FinalAtRiskLog.csv
1104
FinalHighValueReport.csv
1518
FinalOrderDetails.csv
3197
lineage.csv
7349
lineageMetadata.csv
7349
OrderDetails.csv
7349
OrderDetailsDummy.csv
20284
Orders.csv
24435
OrderSummary.csv
36897
Shippers.csv
40218
TableFulfillmentLeads.csv
40232
TableOrderCategories.csv
46043
TempAtRisk.csv
50196
TempHighValue.csv
50537
TempOrderSummary.csv
52214
vwOrderFulfillment.csv
56365
vwOrderTotals.csv
61346


In [2]:
for directory in os.listdir(data_dir):
    if not directory.startswith('lineage.csv'):
        continue
    with open(data_dir + '/' + directory, newline='') as csvfile:
        
        reader = csv.reader(csvfile, delimiter=';')
        csvfile.readline()
        for row in reader:
            table1 = parse_name(row[0])
            column1 = row[1]
            value1 = row[2]
            table2 = parse_name(row[3])
            column2 = row[4]
            value2 = row[5]
            if "'" in value1 or "'" in value2: continue
            query1 = f"""
                          PREFIX : <http://www.semanticweb.org/put/database-data-lineage/>
                          PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                          SELECT ?cellValue 
                          WHERE {{
                              ?x :hasCellValue ?cellValue .
                              ?cellValue :exactValue "{value1}" .
                              ?cellValue :belongsToColumn ?column .
                              ?column rdfs:label "{column1}" .
                              ?table :hasColumn ?column .
                              ?table :title "{table1}" .
                                                        }}
                          """
            query2 = f"""
                          PREFIX : <http://www.semanticweb.org/put/database-data-lineage/>
                          PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                          SELECT ?cellValue 
                          WHERE {{
                              ?x :hasCellValue ?cellValue .
                              ?cellValue :exactValue "{value2}" .
                              ?cellValue :belongsToColumn ?column .
                              ?column rdfs:label "{column2}" .
                              ?table :hasColumn ?column .
                              ?table :title "{table2}" .
                                                        }}
                          """
            result1 = owl.default_world.sparql(query1)
            row1 = result1
            result2 = owl.default_world.sparql(query2)
            row2 = result2
            for r1 in row1:
                for r2 in row2:
                    r1[0].rowDerivedFrom = [r2[0]]

In [ ]:
ontology.save('../ontology/put-dataset-data-lineage-individuals-full.ttl')



In [4]:
result = owl.default_world.sparql(f"""
                          PREFIX : <http://www.semanticweb.org/put/database-data-lineage/>
                          PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                          SELECT ?x 
                          WHERE {{
                              ?x :hasCellValue ?cellValue .
                              ?cellValue :exactValue "11073" .
                              ?column :hasInstance ?cellValue .
                              ?column rdfs:label "OrderID" .
                              ?table :hasColumn ?column .
                              ?table :title "OrderDetails.csv" .
                          }}
                          """)
for r in result:
    print(r)

In [5]:
result = owl.default_world.sparql(f"""
                          PREFIX : <http://www.semanticweb.org/put/database-data-lineage/>
                          PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                          SELECT ?cellValue 
                          WHERE {{
                              ?x :hasCellValue ?cellValue .
                              ?cellValue :exactValue "10667" .
                              ?cellValue :belongsToColumn ?column .
                              ?column rdfs:label "OrderID" .
                              ?table :hasColumn ?column .
                              ?table :title "Orders" .
                                                        }}
                          """)
for r in result:
    print(r)

[put-dataset-data-lineage-individuals-example.cellvalue1984]
[put-dataset-data-lineage-individuals-example.cellvalue4275]
[put-dataset-data-lineage-individuals-example.cellvalue11424]
[put-dataset-data-lineage-individuals-example.cellvalue11429]
[put-dataset-data-lineage-individuals-example.cellvalue18370]
[put-dataset-data-lineage-individuals-example.cellvalue25880]
[put-dataset-data-lineage-individuals-example.cellvalue32891]
[put-dataset-data-lineage-individuals-example.cellvalue40789]
[put-dataset-data-lineage-individuals-example.cellvalue43347]
[put-dataset-data-lineage-individuals-example.cellvalue45638]
[put-dataset-data-lineage-individuals-example.cellvalue49377]
[put-dataset-data-lineage-individuals-example.cellvalue52270]


In [6]:
import owlready2 as owl

result = owl.default_world.sparql(f"""
                          PREFIX : <http://www.semanticweb.org/put/database-data-lineage/>
                          PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                          SELECT ?table 
                          WHERE {{

                              ?table :rowDerivedFrom ?x .
                                                        }}
                          """)
for r in result:
    print(r)

[put-dataset-data-lineage-individuals-base.cellvalue2]
[put-dataset-data-lineage-individuals-base.cellvalue13]
[put-dataset-data-lineage-individuals-base.cellvalue24]
[put-dataset-data-lineage-individuals-base.cellvalue35]
[put-dataset-data-lineage-individuals-base.cellvalue46]
[put-dataset-data-lineage-individuals-base.cellvalue57]
[put-dataset-data-lineage-individuals-base.cellvalue68]
[put-dataset-data-lineage-individuals-base.cellvalue79]
[put-dataset-data-lineage-individuals-base.cellvalue101]
[put-dataset-data-lineage-individuals-base.cellvalue123]
[put-dataset-data-lineage-individuals-base.cellvalue134]
[put-dataset-data-lineage-individuals-base.cellvalue145]
[put-dataset-data-lineage-individuals-base.cellvalue156]
[put-dataset-data-lineage-individuals-base.cellvalue167]
[put-dataset-data-lineage-individuals-base.cellvalue178]
[put-dataset-data-lineage-individuals-base.cellvalue189]
[put-dataset-data-lineage-individuals-base.cellvalue200]
[put-dataset-data-lineage-individuals-ba

In [5]:
train_file = open('../SiaILP/data/lineage/train.txt','w+')
test_file = open('../SiaILP/data/lineage/test.txt','w+')
valid_file = open('../SiaILP/data/lineage/valid.txt','w+')
import random
for individual in ontology.individuals():
    for prop in individual.get_properties():
        for value in prop[individual]:
            value_str = str(value)
            value_str = value_str.replace(' ','_')
            value_str = value_str.replace('put-dataset-data-lineage-individuals-base.','')
            rng = random.random()
            if rng < 0.1:
                train_file.write(f"{individual.name}\t{prop.python_name}\t{value_str}\n")
            elif rng < 0.6 and 'derived' in prop.python_name.lower():
                test_file.write(f"{individual.name}\t{prop.python_name}\t{value_str}\n")
            elif  'derived' in prop.python_name.lower():
                valid_file.write(f"{individual.name}\t{prop.python_name}\t{value_str}\n")
train_file.close()
test_file.close()
valid_file.close()

1104

In [2]:
from rdflib import Graph

g = Graph()
g.parse('../ontology/put-dataset-data-lineage-individuals-base.ttl')
g.serialize(destination='../ontology/put-dataset-data-lineage-individuals-base.rdf', format='xml')

<Graph identifier=Nc042d358652543b581802731e5d6bb8d (<class 'rdflib.graph.Graph'>)>